In [1]:
import pandas as pd
import numpy as np
import math

Ratings=pd.read_csv("ml-latest/ratings.csv")
Movies=pd.read_csv("ml-latest/movies.csv")
Tags=pd.read_csv("ml-latest/tags.csv")

Calculating the TF value and IDF value and multiplying together to get TF-IDF value

In [4]:
TF= Tags.groupby(['movieId','tag'], as_index = False, sort = False).count().rename(columns = {'userId': 'tag_count_TF'})[['movieId','tag','tag_count_TF']]

Tag_distinct = Tags[['tag','movieId']].drop_duplicates()

DF =Tag_distinct.groupby(['tag'], as_index = False, sort = False).count().rename(columns = {'movieId': 'tag_count_DF'})[['tag','tag_count_DF']]

a=math.log10(len(np.unique(Tags['movieId'])))

DF['IDF']=a-np.log10(DF['tag_count_DF'])
                                
TF = pd.merge(TF,DF,on = 'tag', how = 'left', sort = False)
                                
TF['TF-IDF']=TF['tag_count_TF']*TF['IDF']
                                
#Calculating the unit length vector by dividing TF-IDF value with the vector length of a particular movie.

Vect_len=TF[['movieId','TF-IDF']]
                                
Vect_len['TF-IDF-Sq']=Vect_len['TF-IDF']**2
                                
Vect_len =Vect_len.groupby(['movieId'], as_index = False, sort = False).sum().rename(columns = {'TF-IDF-Sq':'TF-IDF-Sq-sum'})[['movieId','TF-IDF-Sq-sum']]

Vect_len['vect_len'] = np.sqrt(Vect_len[['TF-IDF-Sq-sum']].sum(axis=1))
                                
TF = pd.merge(TF,Vect_len,on = 'movieId', how = 'left', sort = False)
                                
TF['TAG_WT']=TF['TF-IDF']/TF['vect_len']


C:\Users\lalwa\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Calculating the unit length vector by dividing TF-IDF value with the vector length of a particular movie.

In [6]:
Vect_len = TF[['movieId','TF-IDF']]

Vect_len['TF-IDF-Sq'] = Vect_len['TF-IDF']**2

Vect_len = Vect_len.groupby(['movieId'], as_index = False, sort = False).sum().rename(columns = {'TF-IDF-Sq': 'TF-IDF-Sq-sum'})[['movieId','TF-IDF-Sq-sum']]

Vect_len['vect_len'] = np.sqrt(Vect_len[['TF-IDF-Sq-sum']].sum(axis=1))

TF = pd.merge(TF,Vect_len,on = 'movieId', how = 'left', sort = False)

TF['TAG_WT'] = TF['TF-IDF']/TF['vect_len']



C:\Users\lalwa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In the first part of the assignment with unweighted user profile, the user profile should be the sum of the item-tag vectors of all items the user has rated positively (>= 3.5 stars). 

Implementing the same and calculate user profile for each user.

In [75]:
Ratings_filter = Ratings[Ratings['rating'] >= 3.5]

distinct_users = np.unique(Ratings['userId'])

user_tag_pref=pd.DataFrame()

i=1

for user in distinct_users[1:2]:
    if i%30 ==0:     
        print("user: ", i , "out of: ", len(distinct_users))

        user_data= Ratings_filter[Ratings_filter['userId']==user]

        user_data = pd.merge(TF,user_data,on = 'movieId', how = 'inner', sort = False)
        user_data1 = user_data.groupby(['tag'], as_index = False, sort = False).sum().rename(columns = {'TAG_WT': 'tag_pref'})[['tag','tag_pref']]
 
        user_data1['user']=user
    
        user_tag_pref = user_tag_pref.append(user_data1, ignore_index=True)
        print(user_tag_pref)
        i=i+1
            



Now we are ready with both the user profile and item profile. Now the final step is to calculate the cosine similarity between the two vectors.

In [63]:
distinct_users = np.unique(Ratings_filter['userId'])

tag_merge_all=pd.DataFrame()

i=1

for user in distinct_users[1:2]:
    user_tag_pref_all = user_tag_pref[user_tag_pref['user'] == user]
    distinct_movies = np.unique(TF['movieId'])
    j=1
    
    for movie in distinct_movies:
        if j%300==0:
 
            print("movie: ", j , "out of: ", len(distinct_movies) , "with user: ", i , "out of: ", len(distinct_users))
 
            TF_Movie= TF[TF['movieId'] == movie]
            tag_merge = pd.merge(TF_Movie,user_tag_pref_all,on = 'tag', how = 'left', sort = False)
            tag_merge['tag_pref']=tag_merge['tag_pref'].fillna(0)
            tag_merge['tag_value']=tag_merge['TAG_WT']*tag_merge['tag_pref']
 
            TAG_WT_val=np.sqrt(np.sum(np.square(tag_merge['TAG_WT']), axis=0))
            tag_pref_val=np.sqrt(np.sum(np.square(user_tag_pref_all['tag_pref']), axis=0))
 
            tag_merge_final = tag_merge.groupby(['user','movieId'])[['tag_value']].sum().rename(columns = {'tag_value':'Rating'}).reset_index()
 
            tag_merge_final['Rating']=tag_merge_final['Rating']/(TAG_WT_val*tag_pref_val)
            tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)
            j=j+1
            i=i+1
            tag_merge_all=tag_merge_all.sort_index(by=['user','Rating']).reset_index()
            print(tag_merge_all)

In [64]:
distinct_users = np.unique(Ratings['userId'])
user_tag_pref = pd.DataFrame()
i=1
for user in distinct_users[1:2]:

    if i%30==0:
        print ("user: ", i , "out of: ", len(distinct_users))
 
user_data = Ratings[Ratings['userId'] == user]
               
user_data['weight'] = user_data['rating']-user_data['rating'].drop_duplicates().mean()
user_data1 = pd.merge(TF,user_data,on = 'movieId', how = 'inner', sort = False)
user_data1['TAG_WT_WTD'] = user_data1['TAG_WT']*user_data1['weight']
user_data2 = user_data1.groupby(['tag'], as_index = False, sort = False).sum().rename(columns = {'TAG_WT_WTD':'tag_pref'})[['tag','tag_pref']]
user_data2['user'] = user
user_tag_pref = user_tag_pref.append(user_data2, ignore_index=True)
i=i+1
print(user_tag_pref)

                                      tag  tag_pref  user
0                                  sci-fi  1.042482     2
1                             space opera  0.395627     2
2                               mythology  0.006269     2
3                                   nerdy  0.021201     2
4                         space adventure  0.382588     2
5                                 fantasy  0.277158     2
6     the classic sci fi movie. must see.  0.022611     2
7                          classic sci-fi  0.418305     2
8                            clever names  0.002261     2
9                                   geeky  0.011866     2
10                             book-based  0.002261     2
11                                  space  0.970338     2
12                          sci-fi, space  0.014697     2
13                           good vs evil  0.145414     2
14                             futuristic  0.050056     2
15                        Science Fiction  0.378804     2
16            

C:\Users\lalwa\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Again calculating cosine similarity between the two vectors

In [61]:
distinct_users=np.unique(Ratings['userId'])

tag_merge_all=pd.DataFrame()

i=1

for user in distinct_users[1:2]:
 
    user_tag_pref_all= user_tag_pref[user_tag_pref['user']==user]
    distinct_movies = np.unique(TF['movieId'])
    j=1
    for movie in distinct_movies:
         if j%300==0:
            print("movie: ", j , "out of: ", len(distinct_movies) , "with user: ", i , "out of: ", len(distinct_users))
 
TF_Movie= TF[TF['movieId'] == movie]
tag_merge = pd.merge(TF_Movie,user_tag_pref_all,on = 'tag', how = 'left', sort = False)

tag_merge['tag_pref']=tag_merge['tag_pref'].fillna(0)

tag_merge['tag_value']=tag_merge['TAG_WT']*tag_merge['tag_pref']
 
TAG_WT_val=np.sqrt(np.sum(np.square(tag_merge['TAG_WT']), axis=0))

tag_pref_val=np.sqrt(np.sum(np.square(user_tag_pref_all['tag_pref']), axis=0))
 
tag_merge_final = tag_merge.groupby(['user','movieId'])[['tag_value']].sum().rename(columns = {'tag_value': 'Rating'}).reset_index()
 
tag_merge_final['Rating']=tag_merge_final['Rating']/(TAG_WT_val*tag_pref_val)
 
tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)

j=j+1
i=i+1
tag_merge_all=tag_merge_all.sort_index(by=['user','Rating']).reset_index()

print(tag_merge_all)

   index  user  movieId  Rating
0      0   2.0   176279     0.0


C:\Users\lalwa\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
